# How to Use Feature Extraction Toolbox

In [1]:
import pandas as pd

from package_name.feature_extraction import settings, extraction

In [2]:
# Load dataset
filename = "/home/scai9/feature_dataset/USCHAD_data.csv"
dataset = pd.read_csv(filename)

# Get subset of dataset to use for testing
dataset = dataset.loc[dataset["subject"]==1]

### Parameter Settings for Feature Extraction

There are three categories of features: Statistical, Spectral, and Time Frequency. Each have a class holding parameters required for feature calculation. A given configuration can be saved in either a json or yaml format.

In [3]:
# Initialization of the feature extraction parameters
statistical_params = settings.StatisticalFeatureParams(25)
spectral_params = settings.SpectralFeatureParams(25)
time_freq_params = settings.TimeFrequencyFeatureParams(25)

# Save and load the parameters
statistical_params.to_json("statistical_params.json")
statistical_params_2 = settings.StatisticalFeatureParams.from_json("statistical_params.json")

assert statistical_params.get_settings_as_dict() == statistical_params_2.get_settings_as_dict()

### Data Format for Feature Calculators

The individual statistial feature calculator functions support univariate series inputs. See example below.

In [4]:
from package_name.feature_extraction.statistical_feature_calculators import calculate_area_under_squared_curve

area = calculate_area_under_squared_curve(dataset["accx"].values)
print(f"Area Under Squared Curve: {area}")

Area Under Squared Curve: 40958.88911775005


### Using Feature Extraction Functions

In `package_name.feature_extraction.extraction` there are functions to extract features from each subcategory: Statistical, Spectral, and Time-Frequency. There is also a fucntion available to extraction features from all three categories. The `signal_name` parameter specifies a name for the dataset to be prepended to all sub-signal names in the resultant dataframe. `njobs` also specifies the number of cores to use. A basic example of each is shown below.


In [ ]:
# Calculate statistical features
features = extraction.calculate_statistical_features(dataset, statistical_params, columns=["accx", "accy", "accz"], signal_name="test", njobs=1)

print(features.head())

In [7]:
# Calculate spectral features
features = extraction.calculate_spectral_features(dataset, spectral_params, columns=["accx", "accy", "accz"], signal_name="test", njobs=1)

print(features.head())

/home/scai9/proj-adl-classification/package_name/feature_extraction/spectral_feature_calculators.py:1462: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  valley_width_mode = mode(valley_widths)[0]


           spectral_centroid_order_1  spectral_centroid_order_2  \
test_accx                   5.141873                  38.604725   
test_accy                   5.759207                  46.973967   
test_accz                   5.790333                  46.604544   

           spectral_centroid_order_3  spectral_centroid_order_4  \
test_accx                 341.026915                3300.370039   
test_accy                 436.952883                4359.183368   
test_accz                 425.692532                4166.225257   

           spectral_centroid_order_5     spectral_variance  spectral_skewness  \
test_accx               33736.373902  [12.165864506310944]           0.410417   
test_accy               45381.461031  [13.805503482397702]           0.144313   
test_accz               42583.239419  [13.076591019536808]           0.093072   

           spectral_kurtosis  median_frequency  spectral_flatness  ...  \
test_accx           2.116443          3.906250           0.7033

In [ ]:
# Calculate time frequency features
features = extraction.calculate_time_frequency_features(dataset, time_freq_params, columns=["accx", "accy", "accz"], signal_name="test", njobs=1)

print(features.head())

Arrays, DataFrames and Series are all acceptable input data formats. If the input is a DataFrame, the columns parameters specifies the columns to analyze (as seen in the previous examples). If not, they are the ordered names of the components of signal. 

In [6]:
# 2D array input
# print(dataset[["accx", "accy", "accz"]].values.shape)
# features = extraction.calculate_statistical_features(dataset[["accx", "accy", "accz"]].values, statistical_params, columns=["accx", "accy", "accz"], signal_name="test", njobs=1)

# print(features.head())

# Series input
features = extraction.calculate_statistical_features(dataset["accx"].values, statistical_params, columns=["accx"], signal_name="test", njobs=1)

print(features.head())

(39076, 3)
Error calculating feature(s) sample_entropy: Unable to allocate 34.1 GiB for an array with shape (39074, 39074, 3) and data type float64
Feature(s) sample_entropy will be excluded.
Error calculating feature(s) approximate_entropy: Unable to allocate 22.8 GiB for an array with shape (39075, 39075, 2) and data type float64
Feature(s) approximate_entropy will be excluded.


/home/scai9/proj-adl-classification/package_name/feature_extraction/statistical_feature_calculators.py:2481: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(signal)[0][0]


Error calculating feature(s) sample_entropy: Unable to allocate 34.1 GiB for an array with shape (39074, 39074, 3) and data type float64
Feature(s) sample_entropy will be excluded.
Error calculating feature(s) approximate_entropy: Unable to allocate 22.8 GiB for an array with shape (39075, 39075, 2) and data type float64
Feature(s) approximate_entropy will be excluded.


/home/scai9/proj-adl-classification/package_name/feature_extraction/statistical_feature_calculators.py:2481: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(signal)[0][0]


Error calculating feature(s) sample_entropy: Unable to allocate 34.1 GiB for an array with shape (39074, 39074, 3) and data type float64
Feature(s) sample_entropy will be excluded.
Error calculating feature(s) approximate_entropy: Unable to allocate 22.8 GiB for an array with shape (39075, 39075, 2) and data type float64
Feature(s) approximate_entropy will be excluded.


/home/scai9/proj-adl-classification/package_name/feature_extraction/statistical_feature_calculators.py:2481: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(signal)[0][0]


               mean  geometric_mean  harmonic_mean  trimmed_mean_0.1  \
test_accx  0.742372        0.907159       0.702253          0.790657   
test_accy  0.188140        0.250165       0.106604          0.211222   
test_accz -0.158577        0.099436       0.027122         -0.152811   

           trimmed_mean_0.15  trimmed_mean_0.2  trimmed_mean_0.25  \
test_accx           0.847635          0.863518           0.872033   
test_accy           0.213097          0.212477           0.210095   
test_accz          -0.148787         -0.144394          -0.141309   

           trimmed_mean_0.3  mean_of_abs  geometric_mean_of_abs  ...  \
test_accx          0.879215     0.936795               0.852483  ...   
test_accy          0.211006     0.311715               0.205384  ...   
test_accz         -0.142072     0.241382               0.162420  ...   

           hurst_exponent  adf_teststats  adf_pvalue  adf_usedlag  \
test_accx        0.803642      -2.638544    0.085290         54.0   
test_ac

AttributeError: 'function' object has no attribute 'names'

In [6]:
statistical_params = settings.StatisticalFeatureParams(25, calculators=["mean", "mode", "std"])

features = extraction.calculate_statistical_features(dataset, statistical_params, columns=["accx", "accy", "accz"], signal_name="test", njobs=1)
print(features.head())

/home/scai9/proj-adl-classification/package_name/feature_extraction/statistical_feature_calculators.py:2481: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(signal)[0][0]


               mean       std      mode
test_accx  0.742372  0.705045  0.864040
test_accy  0.188140  0.342796  0.509207
test_accz -0.158577  0.266645  0.125137
